In [5]:
import pandas as pd
import numpy as np
data = pd.read_csv("Abominable_Data_HW_LABELED_TRAINING_DATA__v772_2231.csv")

# quantising the data
def quant(v):
    return round(v*2)/2

data["Age"] = data["Age"].round()
data["Ht"] = data["Ht"].round()
data["TailLn"] = data["TailLn"].apply(quant)
data["HairLn"] = data["HairLn"].apply(quant)
data["BangLn"] = data["BangLn"].apply(quant)
data["Reach"] = data["Reach"].apply(quant)

# create new features
data["Shagginess"] = data["HairLn"] - data["BangLn"]
data["ApeFactor"] = data["Reach"] - data["Ht"]


### create a test suite of length 8 with attribute Age. -- Test Suite B

# greater than 100 bhuttan and less than assam
x = data[(data["Ht"] <= 100) & (data["ClassID"] == -1 )].head(4)
y = data[(data["Ht"] > 100) & (data["ClassID"] == 1 )].head(4)

z = pd.concat([x,y])
z.to_csv("Test_Suite_A_height.csv", index = False)


# greater than 30 belong to assam and less than 30 belong to 1
x = data[(data["Age"] <= 30) & (data["ClassID"] == 1 )].head(4)
y = data[(data["Age"] > 30) & (data["ClassID"] == 1 )].head(4)
z = pd.concat([x,y])
z.to_csv("Test_Suite_B_age.csv", index = False)

# create a test suite of length 8 with attribute tail length. -- Test Suite C 
x = data[(data["TailLn"] <= 10) & (data["ClassID"] == 1 )].head(4)
y = data[(data["TailLn"] > 10) & (data["ClassID"] == -1 )].head(4)
z = pd.concat([x,y])
z.to_csv("Test_Suite_C_tail.csv", index = False)

In [ ]:
# Finding the best attribute with its best threshold.

data = pd.read_csv("Test_Suite_B_age.csv")
attributes = list(data.columns[:6])

def find_entropy(noden):
    """
    Calcultes entropy for the given node
    """
    e = 0
    for i in noden:
        pv = (i/np.sum(noden))
        if pv == 0:
            e += 0
        else:
            e += (pv * np.log2(pv))
    return -e

def Fgain_ratio(noden,pe):
    """
    Calculates the information gain for the give data.
    """
    s = 0 # average entropy of the set; second term in the gain formula
    spliti = 0 # split info
    parent_entropy = find_entropy(pe) # entropy of the entire set
    
    for i in noden:
        niBYnt = (np.sum(i)/np.sum(noden))
        s +=  niBYnt * find_entropy(i) 
        spliti += niBYnt * np.log2(niBYnt)
    spliti = -spliti
    information_gain = parent_entropy - s
    information_gain_Ratio = information_gain/spliti
    # print(parent_entropy, s, information_gain,spliti,information_gain_Ratio )
    # print("igr", information_gain_Ratio)
    return information_gain_Ratio

dict =  {}
for attribute in attributes:
    # print(attribute)
    best_igr = -1.0
    pe = [len(data[data["ClassID"] == 1]), len(data[data["ClassID"] == -1])]
    for threshold in np.arange(data[attribute].min(), data[attribute].max()+1):
        # splitting the attribute on the given threshold.
        nodeL = data[data[attribute] <= threshold] 
        nodeR = data[data[attribute] > threshold]
        nodeL_splits = [len(nodeL[nodeL["ClassID"] == 1]), len(nodeL[nodeL["ClassID"] == -1])]
        nodeR_splits = [len(nodeR[nodeR["ClassID"] == 1]), len(nodeR[nodeR["ClassID"] == -1])]
        x = [nodeL_splits, nodeR_splits]
        igr = Fgain_ratio(x, pe)
        if igr > best_igr:
            best_igr = igr
            best_thres = threshold
    dict[attribute] = [best_thres, best_igr]
print(dict)


